In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import sys
sys.argv = [sys.argv[0]]

In [21]:
from alphaledger.universe import Universe
from alphaledger.config import settings

universe = Universe(settings.universe_name)
# filings_df = universe.get_filings()

# numeric_df, text_df = process_filing_urls(filings_df)

INFO:alphaledger.universe:Loading universe definition from: /Users/forrest.murray/Documents/alphaledger/universes/sectors/cloud_computing.yaml
INFO:alphaledger.universe:Found existing filings metadata file: /Users/forrest.murray/Documents/alphaledger/output/sec_filings/Cloud_Computing_Giants.delta. Initializing LazyFrame.
/Users/forrest.murray/Documents/alphaledger/src/alphaledger/universe.py:236: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  f"Successfully initialized filings_lf for {self.name}. Schema: {self.filings_lf.schema}"
INFO:alphaledger.universe:Initialized Universe 'Cloud_Computing_Giants' from /Users/forrest.murray/Documents/alphaledger/universes/sectors/cloud_computing.yaml with 16 securities. Filings metadata status: Detected.


In [22]:
universe.securities

{'AMZN': Security(ticker='AMZN', name='Amazon.com, Inc.', exchange='NASDAQ', sector='Consumer Discretionary', industry='Internet Retail', currency='USD', country='US', custom_sector=None, custom_industry=None, subsector=None, theme=None),
 'MSFT': Security(ticker='MSFT', name='Microsoft Corporation', exchange='NASDAQ', sector='Information Technology', industry='Software—Infrastructure', currency='USD', country='US', custom_sector=None, custom_industry=None, subsector=None, theme=None),
 'GOOGL': Security(ticker='GOOGL', name='Alphabet Inc.', exchange='NASDAQ', sector='Communication Services', industry='Internet Content & Information', currency='USD', country='US', custom_sector=None, custom_industry=None, subsector=None, theme=None),
 'IBM': Security(ticker='IBM', name='International Business Machines Corporation', exchange='NYSE', sector='Information Technology', industry='Information Technology Services', currency='USD', country='US', custom_sector=None, custom_industry=None, subsect

In [4]:
# u = Universe("sectors/cloud_computing")   
filings = universe.collect_filings()

NameError: name 'universe' is not defined

In [5]:
filings.collect()

NameError: name 'filings' is not defined

In [ ]:
import polars as pl
filings.filter(pl.col("cik").is_in(["0001561550"])).collect()

In [23]:
amz_facts = universe.get_security_numeric_facts(ticker="AMZN")

INFO:alphaledger.process_xbrl:[process_filing_urls_direct] Starting direct fact extraction.


ERROR:alphaledger.sec:Error parsing XBRL for AMZN (Filing Date: 2016-01-29): not well-formed (invalid token): line 7, column 2
Traceback (most recent call last):
  File "/Users/forrest.murray/Documents/alphaledger/src/alphaledger/sec.py", line 924, in parse_filing_xbrl
    inst = parser.parse_instance(schema_url)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/forrest.murray/Documents/alphaledger/.venv/lib/python3.11/site-packages/xbrl/instance.py", line 811, in parse_instance
    return parse_ixbrl_url(uri, self.cache) if is_url(uri) else parse_ixbrl(uri, self.cache, instance_url, encoding)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/forrest.murray/Documents/alphaledger/.venv/lib/python3.11/site-packages/xbrl/instance.py", line 481, in parse_ixbrl_url
    return parse_ixbrl(instance_path, cache, instance_url, encoding)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/forrest.murray/Documents/alphaledger/.venv/lib/python3.11

In [ ]:
amz_facts.collect()

In [6]:
from alphaledger.config import settings

settings.verbose = True
ibm_facts = universe.get_security_numeric_facts("IBM").collect()

INFO:alphaledger.universe:Data file found: Existing numeric facts for universe 'Cloud_Computing_Giants' at /Users/forrest.murray/Documents/alphaledger/output/numeric_facts/Cloud_Computing_Giants.delta. Returning LazyFrame.
INFO:alphaledger.universe:Processing numeric facts for universe 'Cloud_Computing_Giants' using direct method...
/Users/forrest.murray/Documents/alphaledger/src/alphaledger/universe.py:858: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  col in filings_lf.columns for col in required_cols_in_filings_lf
INFO:alphaledger.universe:Found 131 filings for universe 'Cloud_Computing_Giants'. Processing XBRL using direct method...
INFO:alphaledger.process_xbrl:[process_filing_urls_direct] Starting direct fact extraction.
ERROR:alphaledger.process_xbrl:[direct] ERROR processing https://www.sec.gov/Arch

In [ ]:
ibm_facts

In [ ]:
import polars as pl

# universe.fetch_or_load_filings()

# numeric_df.filter(pl.col("context_entity").is_in(["0001018724"]))\

# numeric_df.head(1)

In [ ]:
num_df = universe.get_numeric_facts()

In [ ]:
num_df.collect().count()

In [ ]:
num_df.collect().head(1)

In [ ]:
universe.get_all_securities()